<div class="align-center">
<a href="https://oumi.ai/"><img src="https://oumi.ai/docs/en/latest/_static/logo/header_logo.png" height="200"></a>

[![Documentation](https://img.shields.io/badge/Documentation-latest-blue.svg)](https://oumi.ai/docs/en/latest/index.html)
[![Discord](https://img.shields.io/discord/1286348126797430814?label=Discord)](https://discord.gg/oumi)
[![GitHub Repo stars](https://img.shields.io/github/stars/oumi-ai/oumi)](https://github.com/oumi-ai/oumi)
</div>

👋 Welcome to Open Universal Machine Intelligence (Oumi)!

🚀 Oumi is a fully open-source platform that streamlines the entire lifecycle of foundation models - from [data preparation](https://oumi.ai/docs/en/latest/resources/datasets/datasets.html) and [training](https://oumi.ai/docs/en/latest/user_guides/train/train.html) to [evaluation](https://oumi.ai/docs/en/latest/user_guides/evaluate/evaluate.html) and [deployment](https://oumi.ai/docs/en/latest/user_guides/launch/launch.html). Whether you're developing on a laptop, launching large scale experiments on a cluster, or deploying models in production, Oumi provides the tools and workflows you need.

🤝 Make sure to join our [Discord community](https://discord.gg/oumi) to get help, share your experiences, and contribute to the project! If you are interested in joining one of the community's open-science efforts, check out our [open collaboration](https://oumi.ai/community) page.

⭐ If you like Oumi and you would like to support it, please give it a star on [GitHub](https://github.com/oumi-ai/oumi).

# vLLM Inference Engine

This notebook demonstrates how to use the `VLLMInferenceEngine` class for inference with Llama 3.3 70B.

# Prerequisites

## Machine Requirements

❗**NOTICE:** This notebook doesn't run on Colab because the GPU is too old to be supported by vLLM.

It is recommended to run this notebook on a machine with GPU support, as vLLM is mainly intended to run on GPUs. Llama 3.3 70B requires 140GB VRAM to serve, though we also provide examples below for inference with Llama 3.1 8B, Llama 3.2 1B, and quantized Llama 3.3 70B that require less memory.

If your local machine cannot run this notebook, you can instead run this notebook on a cloud platform. The following demonstrates how to open a VSCode instance backed by a GCP node with 4 A100 GPUs, from which the notebook can be run.

```bash
# Run on your local machine
gcloud auth application-default login  # Authenticate with GCP
make gcpcode ARGS="--resources.accelerators A100:4"  # 4 A100-40GB GPUs, enough for 70B model. Can also use 2x "A100-80GB"
```

## Oumi Installation

First, let's install Oumi and vLLM. You can find more detailed instructions about Oumi installation [here](https://oumi.ai/docs/en/latest/get_started/installation.html). Here, we include Oumi's GPU dependencies.


In [1]:
%pip install oumi[gpu]

## Llama Access

Llama 3.3 70B is a gated model on HuggingFace Hub. To run this notebook, you must first complete the [agreement](https://huggingface.co/meta-llama/Llama-3.3-70B-Instruct) on HuggingFace, and wait for it to be accepted. Then, specify `HF_TOKEN` below to enable access to the model if it's not already set.

Usually, you can get the token by running this command `cat ~/.cache/huggingface/token` on your local machine.

In [2]:
import os

if not os.environ.get("HF_TOKEN"):
    # NOTE: Set your Hugging Face token here if not already set.
    os.environ["HF_TOKEN"] = "<MY_HF_TOKEN>"
hf_token = os.environ.get("HF_TOKEN")
print(f"Using HF Token: '{hf_token}'")

# This is needed for vLLM to use multiple GPUs in a notebook.
# If you're not running in a notebook, you can ignore this.
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"

To download Llama 3.3 70B to your machine before inference, run:

In [3]:
%pip install hf_transfer
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
! hf download meta-llama/Llama-3.3-70B-Instruct --exclude original/*

In [4]:
import torch

from oumi.core.configs import InferenceConfig
from oumi.core.types import Conversation, Message, Role
from oumi.inference import VLLMInferenceEngine

In [5]:
# If we have multiple GPUs, we can use Ray to parallelize the inference.
# This is essential if you're running a model that's too big to fit in a single GPU.

import ray

if torch.cuda.is_available() and torch.cuda.device_count() >= 2:
    ray.shutdown()
    ray.init(num_gpus=torch.cuda.device_count())

### Setting up the config file

Note: in this section we are writing the config file to the current working directory.

An alternative option is to initialize the params classes directly: `ModelParams`, `GenerationParams`.

In [6]:
config_path = "vllm_tutorial_llama70b_infer.yaml"

In [7]:
%%writefile vllm_tutorial_llama70b_infer.yaml

model:
  # model_name: "meta-llama/Llama-3.1-8B-Instruct"  # 8B model, requires 1x A100-40GB GPUs
  model_name: "meta-llama/Llama-3.3-70B-Instruct"  # 70B model, requires 4x A100-40GB GPUs
  model_max_length: 512
  torch_dtype_str: "bfloat16"
  trust_remote_code: True
  attn_implementation: "sdpa"

generation:
  max_new_tokens: 128
  batch_size: 1

### Load the model and the inference engine

In [8]:
%%time

# Download, and load the model in memory
# This may take a while, depending on your internet speed.
# The inference engine only needs to be loaded once and can be
# reused for multiple conversations.

config = InferenceConfig.from_yaml(config_path)

inference_engine = VLLMInferenceEngine(
    config.model,
    tensor_parallel_size=torch.cuda.device_count(),  # use all available GPUs
    # Adjustments to help Llama-3.3-70B-Instruct run on 4 A100-40GB GPUs
    enable_prefix_caching=False,
    gpu_memory_utilization=0.95,
    max_num_seqs=10,
)

### Preprocessing our inputs

The inference engine expects a list of conversations, where each conversation is a list of messages.

See the [Conversation](https://github.com/oumi-ai/oumi/blob/38b3d2b27407be5fc9be5a1dd88f9ad518f3491c/src/oumi/core/types/turn.py#L109) class for more details.

Tip: you can visualize how the conversation is rendered as a prompt with the following:

```python
inference_engine.apply_chat_template(conversation, tokenize=False)
```

In [9]:
conversations = [
    Conversation(
        messages=[
            Message(
                role=Role.SYSTEM, content="Translate the following text into French."
            ),
            Message(role=Role.USER, content="Hello, how are you?"),
        ]
    ),
]

### Running inference

Under the hood, the vLLM engine will batch the conversations to run inference with a high throughput.

Make sure to feed all your prompts to the engine at once for maximum throughput.

In [10]:
%%time

print(f"Running inference for {len(conversations)} conversations")

generations = inference_engine.infer(
    input=conversations,
    inference_config=config,
)

In [11]:
for conversation in generations:
    print(repr(conversation))
    print()

### Bonus: Running quantized GGUF models

You can also run quantized GGUF models, by downloading the model file and passing it to the engine.

For example, to run the Llama 3.3 70B model quantized at 4-bit, you can do the following: 

First, we download the GGUF model file. There are multiple quantization schemes available, here we choose the `Q4_K_S` scheme which is 4-bit with the `K_S` quantization algorithm.

In [12]:
from huggingface_hub import hf_hub_download

repo_id = "bartowski/Llama-3.3-70B-Instruct-GGUF"
filename = "Llama-3.3-70B-Instruct-Q4_K_S.gguf"

# Will download the model in the current working directory instead of HF_CACHE_DIR
model_path = hf_hub_download(repo_id, filename=filename, local_dir=".")

We then update the config file to point to the model we just downloaded:

In [13]:
%%writefile vllm_tutorial_llama70b_infer.yaml

model:
  # Filepath to the GGUF model, which we just downloaded, see `model_path` output above
  model_name: "Meta-Llama-3.1-70B-Instruct-Q4_K_S.gguf"  
  # GGUF files do not have a config. We need to specify the tokenizer name manually.
  tokenizer_name: "meta-llama/Llama-3.3-70B-Instruct"  
  model_max_length: 512
  torch_dtype_str: "float16"  # GGUF models require float16
  trust_remote_code: True
  attn_implementation: "sdpa"

generation:
  max_new_tokens: 128
  batch_size: 1